# Positive samples for the dataset

## Extracting relevant information from the dataset

In [2]:
import pandas as pd
import numpy as np
import pydicom
import png
import math
from PIL import Image
from pathlib import Path
import boto3
import matplotlib.pyplot as plt

data_path = '/home/szelesteya/projects/EMBED_Open_Data/'
tables_path = data_path + 'tables/'
image_root_path = '/media/szelesteya/F824D4D024D492CC/EMBED-images/'
image_dcm_path = image_root_path + 'dicom-positive/'

In [3]:
cli_df = pd.read_csv(tables_path + 'EMBED_OpenData_clinical.csv', low_memory=False)

# Only keeping result BIRADS-1 and BIRADS-2 screenings
pos_cli_df = cli_df[(cli_df.asses.isin(['B','S','M','K'])) &
                    ((cli_df.calcfind.notna()) | 
                     (cli_df.asses.isin(['S','M','K'])))][['Unnamed: 0',
                                                           'empi_anon',
                                                           'acc_anon',
                                                           'side',
                                                           'calcfind',
                                                           'calcdistri',
                                                           'otherfind',
                                                           'numfind',
                                                           'path_severity',
                                                           'age_at_study',
                                                           'ETHNICITY_DESC',
                                                           'study_date_anon',
                                                           'asses']]

# Rename columns to prepare for merge
pos_cli_df = pos_cli_df.rename(columns={'study_date_anon':'diag_study_date'})

In [4]:
# Reading image metadata
meta_df = (pd.read_csv(tables_path + 'EMBED_OpenData_metadata_reduced.csv', low_memory=False))
meta_red_df = meta_df[  (meta_df['ROI_coords'] != "()") &
                        (meta_df['FinalImageType'] == '2D') &                         
                        (meta_df['spot_mag'] != 1)][[  'empi_anon',
                                                                'acc_anon',
                                                                'ImageLateralityFinal',
                                                                'anon_dicom_path',
                                                                'study_date_anon',
                                                                'ViewPosition', 
                                                                'num_roi',
                                                                'ROI_coords',
                                                                'spot_mag']]

# Rename columns to prepare for merge
meta_red_ren_df = meta_red_df.rename(columns={'ImageLateralityFinal':'side'})

In [5]:
# Merging clinical information with medical ones
pos_full_df = pos_cli_df.merge(meta_red_ren_df, on=['empi_anon','acc_anon','side'])

# Generate paths for png extraction
pos_full_df['relative_dcm_path'] = pos_full_df['anon_dicom_path'].apply(lambda x: '/'.join(x.split('/')[5:]))

# Keeping relevant columns
pos_empi_df = pos_full_df[['empi_anon',
                           'acc_anon',
                           'side',
                           'asses',
                           'age_at_study',
                           'calcfind',
                           'calcdistri',
                           'otherfind',
                           'numfind',
                           'ViewPosition',
                           'num_roi',
                           'ROI_coords',
                           'ETHNICITY_DESC',
                           'study_date_anon',
                           'diag_study_date',
                           'relative_dcm_path',
                           'spot_mag']]

# Rename columns to be more consistent
pos_empi_df = pos_empi_df.rename(columns={'ETHNICITY_DESC':'eth_desc',
                                          'calcfind':'calc_find',
                                          'calcdistri':'calc_distrib',
                                          'otherfind':'other_find',                                          
                                          'ViewPosition':'view_pos',
                                          'numfind':'num_find'})


# Convert study date so it is easily interpreted by Python
pos_empi_df['diag_study_date'] = pd.to_datetime(pos_empi_df['diag_study_date'], errors='coerce', format= '%Y-%m-%d')
pos_empi_df['study_date_anon'] = pd.to_datetime(pos_empi_df['study_date_anon'], errors='coerce')

# Keeping only screening exams with less than 180 day differential between diagnosis date and last exam date (the diagnosis might be outdated on other circumstances)
pos_empi_df['diag_date_diff'] = pos_empi_df.diag_study_date - pos_empi_df.study_date_anon
pos_empi_rel_df= pos_empi_df.loc[(pos_empi_df.diag_date_diff.dt.days >= 0) & 
                                 (pos_empi_df.diag_date_diff.dt.days <= 180)]

In [67]:
pos_empi_rel_df

,empi_anon,acc_anon,side,asses,age_at_study,calc_find,calc_distrib,other_find,num_find,view_pos,num_roi,ROI_coords,eth_desc,study_date_anon,diag_study_date,relative_dcm_path,spot_mag,diag_date_diff
0,38591157,5821249626639895,R,S,61.931456,"A,K",G,NaN,1,ML,1,"((1610, 1045, 1979, 1463),)",Caucasian or White,2015-07-28,2015-07-28,cohort_1/38591157/1.2.846.113979.3.62.1.506522...,NaN,0 days
1,81221471,9107681375572026,R,M,51.261833,NaN,NaN,NaN,1,CC,1,"((774, 336, 1621, 1062),)",Caucasian or White,2015-03-10,2015-03-10,cohort_1/81221471/1.2.842.113973.3.66.1.510553...,NaN,0 days
2,69442705,3307161731730050,L,M,NaN,NaN,NaN,N,1,CC,1,"((1694, 860, 2298, 1658),)",NaN,2016-09-12,2016-09-12,cohort_1/69442705/1.2.846.113971.3.57.1.541888...,NaN,0 days
3,69442705,3307161731730050,L,M,NaN,NaN,NaN,N,1,MLO,1,"((1708, 1157, 2411, 1768),)",NaN,2016-09-12,2016-09-12,cohort_1/69442705/1.2.846.113971.3.57.1.541888...,NaN,0 days
4,69442705,3307161731730050,L,M,NaN,NaN,NaN,N,1,CC,1,"((1694, 860, 2298, 1658),)",NaN,2016-09-12,2016-09-12,cohort_1/69442705/1.2.846.113971.3.57.1.541888...,NaN,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,69001424,5173595322425404,L,S,69.844008,A,G,NaN,3,ML,1,"((1354, 1338, 1533, 1545),)",Caucasian or White,2020-06-05,2020-06-05,cohort_2/69001424/1.2.847.113979.3.63.1.609325...,NaN,0 days
161,69001424,5173595322425404,L,S,69.844008,A,G,NaN,3,ML,1,"((1354, 1338, 1533, 1545),)",Caucasian or White,2020-06-05,2020-06-05,cohort_2/69001424/1.2.847.113979.3.63.1.609325...,NaN,0 days
162,90551907,7632750486827326,L,S,45.457470,A,G,NaN,2,CC,1,"((2400, 762, 2722, 1113),)",Asian,2020-05-26,2020-05-26,cohort_2/90551907/1.2.844.113970.3.62.1.612437...,NaN,0 days
163,90551907,7632750486827326,L,S,45.457470,A,G,NaN,2,MLO,1,"((2015, 822, 2257, 1149),)",Asian,2020-05-26,2020-05-26,cohort_2/90551907/1.2.844.113970.3.62.1.612437...,NaN,0 days


In [6]:
with open(data_path + 'positive_empirical.csv', 'w') as f:
    pos_empi_rel_df.to_csv(f)

## Pulling the DICOM images with a bash script

In [7]:
# Printing the path to file to read it in the bash script
with open(data_path + 'positive_path.csv', 'w') as f:
    pos_empi_rel_df.drop_duplicates(subset=['relative_dcm_path'])['relative_dcm_path'].to_csv(f, index=False)

In [9]:
%%bash -s "$image_dcm_path" "{data_path}positive_path.csv"

# Pulling dicom files with AWS CLI (Python API didn't work)
dcm_dest_path="$1"
dcm_paths="$2"
ind=$((1))

tail -n +2 $dcm_paths | while IFS= read -r line; do
    relative_path=$(echo "$line" | awk -v OFS='/' '{$1=$1; print}')
    dcm_name=$(echo "$relative_path" | cut -d '/' -f 3-)
               
    file="${dcm_dest_path}$relative_path"
    dir=$(dirname $file)
    mkdir $dir -p
    echo "$ind / 146"
                    
    if [ -f "$file" ]; then
        echo "File already present"
    else
        if [ -f "${dcm_dest_path}$dcm_name" ]; then
            dcm="${dcm_dest_path}$dcm_name"
            dir_dcm=$(dirname $dcm)
            echo "Moving file from ${dir_dcm}"
            mv "${dcm}" "${dcm_dest_path}$relative_path"
        else
            echo "Pulling file $file"
            aws s3 cp "s3://embed-dataset-open/images/$relative_path" "${file}" --profile my-dev-profile
        fi
    fi
    

    ind=$((ind+1))
    clear
done

1 / 146
File already present
2 / 146
File already present
3 / 146
File already present
4 / 146
File already present
5 / 146
File already present
6 / 146
File already present
7 / 146
File already present
8 / 146
File already present
9 / 146
File already present
10 / 146
File already present
11 / 146
File already present
12 / 146
File already present
13 / 146
File already present
14 / 146
File already present
15 / 146
File already present
16 / 146
File already present
17 / 146
File already present
18 / 146
File already present
19 / 146
File already present
20 / 146
File already present
21 / 146
File already present
22 / 146
File already present
23 / 146
File already present
24 / 146
File already present
25 / 146
File already present
26 / 146
File already present
27 / 146
File already present
28 / 146
File already present
29 / 146
File already present
30 / 146
File already present
31 / 146
File already present
32 / 146
File already present
33 / 146
File already present
34 / 146
File alrea

### Converting the DICOM images to PNG

In [8]:
# Get DICOM image metadata
class DCM_Tags():
    def __init__(self, img_dcm):
        try:
            self.laterality = img_dcm.ImageLaterality
        except AttributeError:
            self.laterality = np.nan
            
        try:
            self.view = img_dcm.ViewPosition
        except AttributeError:
            self.view = np.nan
            
        try:
            self.orientation = img_dcm.PatientOrientation
        except AttributeError:
            self.orientation = np.nan

# Check whether DICOM should be flipped
def check_dcm(imgdcm):
    # Get DICOM metadata
    tags = DCM_Tags(imgdcm)
    
    # If image orientation tag is defined
    if ~pd.isnull(tags.orientation):
        # CC view
        if tags.view == 'CC':
            if tags.orientation[0] == 'P':
                flipHorz = True
            else:
                flipHorz = False
            
            if (tags.laterality == 'L') & (tags.orientation[1] == 'L'):
                flipVert = True
            elif (tags.laterality == 'R') & (tags.orientation[1] == 'R'):
                flipVert = True
            else:
                flipVert = False
        
        # MLO or ML views
        elif (tags.view == 'MLO') | (tags.view == 'ML'):
            if tags.orientation[0] == 'P':
                flipHorz = True
            else:
                flipHorz = False
            
            if (tags.laterality == 'L') & ((tags.orientation[1] == 'H') | (tags.orientation[1] == 'HL')):
                flipVert = True
            elif (tags.laterality == 'R') & ((tags.orientation[1] == 'H') | (tags.orientation[1] == 'HR')):
                flipVert = True
            else:
                flipVert = False
        
        # Unrecognized view
        else:
            flipHorz = False
            flipVert = False
            
    # If image orientation tag is undefined
    else:
        # Flip RCC, RML, and RMLO images
        if (tags.laterality == 'R') & ((tags.view == 'CC') | (tags.view == 'ML') | (tags.view == 'MLO')):
            flipHorz = True
            flipVert = False
        else:
            flipHorz = False
            flipVert = False
            
    return flipHorz, flipVert

# Rescale the intensity of the image to get heterogene images with the bit depth of 14
def rescale_to_8bit(image_array):
    upper_percentile = np.percentile(image_array.flatten(), 98) # original_max = np.max(image_array)
    lower_percentile = np.percentile(image_array.flatten(), 2) # original_min = np.min(image_array)
    # max_on_14bit = 16383
    max = 255
    rescaled_array = (image_array - lower_percentile) / (upper_percentile - lower_percentile)
    rescaled_array[rescaled_array < 0] = 0
    rescaled_array[rescaled_array > 1] = 1
    # rescaled_array = np.round((image_array - original_min) / (original_max - original_min) * max_on_14bit).astype(int)
    return np.round(rescaled_array * 255).astype(np.uint8)

def generate_png_path(dcm_path):
    # Get new file name
    split_fn = dcm_path[:-4].split('/')
    new_fn = f"{split_fn[-1]}_conv.png"
    return image_path + new_fn

# Save DICOM pixel array as PNG
def save_dcm_image_as_png(image, png_filename, bitdepth=8):
    with open(png_filename, 'wb') as f:
        rescaled = rescale_to_8bit(image)
        writer = png.Writer(height=rescaled.shape[0], 
                            width=rescaled.shape[1], 
                            bitdepth=bitdepth, 
                            greyscale=True)
        writer.write(f, rescaled.tolist())

def generate_png_path(acc_anon, png_dir):
    # Get new file name
    new_fn = f"{acc_anon}_neg_conv.png"
    return f'{png_dir}/{new_fn}'

# Convert list of DICOMs to PNGs
def process_dcm_list(dcm_list, png_list):    
    for i, dcm_path in enumerate(dcm_list):    
        if not Path(png_list[i]).exists():
            print(f"Processing DICOM #{i}...")
            
            # Load DICOM
            dcm = pydicom.dcmread(dcm_path)
            img = dcm.pixel_array
            
            # Check if a horizontal flip is necessary
            horz, _ = check_dcm(dcm)
            if horz:
                # Flip img horizontally
                img = np.fliplr(img)
            
            # Save PNG            
            save_dcm_image_as_png(img, png_list[i])

def extract_images(data_file_name, dcm_dir, png_dir):
    # Provide a list of DICOM paths and a target directory
    dcm_list = []
    df = pd.read_csv(data_file_name)
    
    for index, row in df.iterrows():
        path = dcm_dir  + row['relative_dcm_path']
        if Path(path).exists():
            dcm_list.append(path)
        
    # Insert png path
    df.loc[:,'png_path'] = df['acc_anon'].apply(lambda x: generate_png_path(x, png_dir))

    # Convert DICOMs
    process_dcm_list(dcm_list, df['png_path'])

    return df

In [9]:
pos_img_emp = extract_images(data_path + 'positive_empirical.csv', image_dcm_path, image_png_path)

with open(data_path + 'positive_empirical_png.csv', 'w') as f:
    (pos_img_emp).to_csv(f, index=False)

Processing DICOM #0...
Processing DICOM #1...
Processing DICOM #2...
Processing DICOM #6...
Processing DICOM #8...
Processing DICOM #9...
Processing DICOM #10...
Processing DICOM #11...
Processing DICOM #13...
Processing DICOM #17...
Processing DICOM #19...
Processing DICOM #20...
Processing DICOM #21...
Processing DICOM #23...
Processing DICOM #25...
Processing DICOM #28...
Processing DICOM #32...
Processing DICOM #34...
Processing DICOM #35...
Processing DICOM #37...
Processing DICOM #38...
Processing DICOM #42...
Processing DICOM #43...
Processing DICOM #46...
Processing DICOM #48...
Processing DICOM #50...
Processing DICOM #51...
Processing DICOM #52...
Processing DICOM #53...
Processing DICOM #55...
Processing DICOM #56...
Processing DICOM #57...
Processing DICOM #59...
Processing DICOM #61...
Processing DICOM #63...
Processing DICOM #65...
Processing DICOM #66...
Processing DICOM #68...
Processing DICOM #70...
Processing DICOM #71...
Processing DICOM #72...
Processing DICOM #73..